# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,13.79,78,21,1.34,NZ,1738566108
1,1,khandyga,62.6667,135.6000,-40.78,99,100,0.55,RU,1738566109
2,2,talnakh,69.4865,88.3972,-23.37,95,100,2.45,RU,1738566110
3,3,kaa-khem,51.6833,94.7333,-13.83,91,100,1.25,RU,1738566112
4,4,krasnovishersk,60.4078,57.0820,-6.79,94,100,3.80,RU,1738566113


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
#%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500, 
    size = "Humidity", 
    scale = 1,
    color = "City",
    alpha = 0.5  
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
# Ideal conditions Max Temp > 21C, Max Temp < 27C, Wind Speed < 4.5 m/s, and 0 cloudiness
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27)]
city_data_df = city_data_df.loc[city_data_df["Wind Speed"] < 4.5]
city_data_df = city_data_df.loc[city_data_df["Cloudiness"] == 0]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,44,coahuayana de hidalgo,18.7000,-103.6583,22.31,45,0,1.63,MX,1738566160
66,66,bredasdorp,-34.5322,20.0403,21.83,53,0,1.75,ZA,1738566187
80,80,ingeniero guillermo n. juarez,-23.9000,-61.8500,25.53,46,0,3.05,AR,1738566204
91,91,suncho corral,-27.9336,-63.4294,25.44,39,0,4.20,AR,1738566218
99,99,rafaela,-31.2503,-61.4867,24.99,44,0,0.45,AR,1738566227
139,139,rajanpur,29.1035,70.3250,23.52,14,0,2.65,PK,1738566275
149,149,pisco,-13.7000,-76.2167,22.03,78,0,1.54,PE,1738566288
150,150,las lomitas,-24.7096,-60.5930,24.55,45,0,2.27,AR,1738566289
154,154,nova vicosa,-17.8919,-39.3719,24.20,92,0,2.87,BR,1738566293
180,180,los frentones,-26.4077,-61.4137,23.61,47,0,2.47,AR,1738566325


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(city_data_df, columns = ["City", "Country", "Lat", "Lng", "Humidity"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
44,coahuayana de hidalgo,MX,18.7000,-103.6583,45,
66,bredasdorp,ZA,-34.5322,20.0403,53,
80,ingeniero guillermo n. juarez,AR,-23.9000,-61.8500,46,
91,suncho corral,AR,-27.9336,-63.4294,39,
99,rafaela,AR,-31.2503,-61.4867,44,
139,rajanpur,PK,29.1035,70.3250,14,
149,pisco,PE,-13.7000,-76.2167,78,
150,las lomitas,AR,-24.7096,-60.5930,45,
154,nova vicosa,BR,-17.8919,-39.3719,92,
180,los frentones,AR,-26.4077,-61.4137,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
coahuayana de hidalgo - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
ingeniero guillermo n. juarez - nearest hotel: Hotel El Indiano
suncho corral - nearest hotel: No hotel found
rafaela - nearest hotel: Parra Hotel & Suites
rajanpur - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
las lomitas - nearest hotel: Hotel Lomitas
nova vicosa - nearest hotel: Pousada Pontal da Barra
los frentones - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
hermanus - nearest hotel: Aloe guest house
kruisfontein - nearest hotel: No hotel found
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
avellaneda - nearest hotel: Hotel Avellaneda
prado - nearest hotel: Pousada Recanto do Prado
port alfred - nearest hotel: No hotel found
ilheus - nearest hotel: No hotel found
plettenberg bay - nearest hotel: Milkwood Manor
concordia - nearest hotel: D'

,City,Country,Lat,Lng,Humidity,Hotel Name
44,coahuayana de hidalgo,MX,18.7000,-103.6583,45,No hotel found
66,bredasdorp,ZA,-34.5322,20.0403,53,Victoria Hotel
80,ingeniero guillermo n. juarez,AR,-23.9000,-61.8500,46,Hotel El Indiano
91,suncho corral,AR,-27.9336,-63.4294,39,No hotel found
99,rafaela,AR,-31.2503,-61.4867,44,Parra Hotel & Suites
139,rajanpur,PK,29.1035,70.3250,14,No hotel found
149,pisco,PE,-13.7000,-76.2167,78,La Portada
150,las lomitas,AR,-24.7096,-60.5930,45,Hotel Lomitas
154,nova vicosa,BR,-17.8919,-39.3719,92,Pousada Pontal da Barra
180,los frentones,AR,-26.4077,-61.4137,47,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
#%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500, 
    size = "Humidity", 
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"],
    alpha = 0.5  
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)